In [ ]:
#@title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License.

# Monitor wildlife health via image classification

🦜🦧 🐘🦩🦋 🦇 🦀 🦔 🐇 🦫 🦏 🐆 🐗 🦙 🦥 🦎 🦃

- **Time estimate**: 2 hours 15 minutes
- **Cost estimate**: less than \$30.00 (_free_ if you use \$300 [Cloud credits](https://cloud.google.com/free/docs/gcp-free-tier))

> [Watch the video in YouTube<br> ![thumbnail](http://img.youtube.com/vi/hUzODH3uGg0/0.jpg)](https://youtu.be/hUzODH3uGg0)
>
> 📜 Read more about this sample in the [blog post](https://cloud.google.com/blog/topics/developers-practitioners/recovering-global-wildlife-populations-using-ml).

This is an interactive notebook that contains all of the code necessary to train an ML model for image classification.
This model is trained to recognize animal species from
[camera trap](https://en.wikipedia.org/wiki/Camera_trap)
pictures.

## 🙈 Using this interactive notebook

Click the **run** icons ▶️ of each section within this notebook. 

This notebook code lets you train and deploy an ML model, as well as test its accuracy, from end-to-end. When you run a code cell, the code runs in the notebook's runtime, so you're not making any changes to your personal computer.

> 🛎️ **To avoid any errors**, wait for each section to finish in their order before clicking the next “run” icon.

This sample must be connected to a Google Cloud project, but nothing else is needed other than your Google Cloud project.
You can use an existing project and the cost will be less than \$30. Alternatively, you can create a new Cloud project with cloud credits for free

## 🚴‍♀️ Steps included

The notebook leverages camera trap images dataset from the
[WCS Camera Traps dataset](http://lila.science/datasets/wcscameratraps)
from the _Labeled Information Library of Alexandria: Biology and Conservation_ (LILA) organization.
 
Here's a quick summary of what you’ll go through:

1. **Create a database of image metadata** _(~5 minutes to complete, costs a few cents)_:
   in a
   [BigQuery](https://cloud.google.com/bigquery)
   table with all the image file names along with their respective category. 

1. **Train an image classifier** _(~2 hours to complete, costs ~$25.00)_:
   With some desired criteria, a
   [Dataflow](https://cloud.google.com/dataflow)
   pipeline creates a balanced dataset from the metadata database.
   It then physically downloads the necessary images from LILA into
   [Cloud Storage](https://cloud.google.com/storage),
   imports the data into
   [AI Platform Unified](https://cloud.google.com/ai-platform-unified/docs/start/introduction-unified-platform),
   and starts the model training job.

1. **Deploy an ML model** _(costs **$1.25 for every hour** the model is deployed online… so complete the “delete model” step soon)_:
   After the model finishes training, we look at the results and deploy it into a Cloud endpoint in AI Platform Unified.

1. **Classify images with the model**:
   We send some images to the model and get back the model's classification predictions.

1. (Optional) **Delete the project** to avoid ongoing costs.

## ✨ Before you begin, you need to…

1. Decide on creating a new
   [free project](https://cloud.google.com/free/docs/gcp-free-tier)
   _(recommended)_ or using an existing one.
   Then copy the project ID and paste it in the `google_cloud_project` field in the "Entering project details” section below.

   > 💡 If you _don't plan to keep the resources_ that you create via this sample, we recommend creating a new project instead of selecting an existing project.
   > After you finish these steps, you can delete the project, removing all the resources associated in bulk.

1. [_Click here_](https://console.cloud.google.com/flows/enableapi?apiid=dataflow,aiplatform.googleapis.com)
   to **enable the following APIs** in your Google Cloud project:
   _Dataflow_ and _AI Platform_.

1. Make sure that **billing is enabled** for your Google Cloud project,
   [_click here_](https://cloud.google.com/billing/docs/how-to/modify-project)
   to learn how to confirm that billing is enabled.

1. [_Click here_](https://console.cloud.google.com/storage/create-bucket)
   to create a Cloud Storage bucket.
   Then copy the bucket’s name and paste it in the `cloud_storage_bucket` field in the “Entering project details” section below.

   > 🛎️ Make sure it's a _regional_ bucket in a location where
   > [AI Platform is available](https://cloud.google.com/ai-platform-unified/docs/general/locations#available_regions).

1. [Create a BigQuery dataset](https://cloud.google.com/bigquery/docs/datasets#create-dataset).
   Then copy the dataset ID and paste it in the `bigquery_dataset` field in the “Entering project details” section below.

   > 💡 You can use the name `samples` for your dataset.

### ⛏️ Preparing the project environment

Click run ▶️ for the following cells to download and install the necessary libraries and resources for this solution.

> 💡 You can _optionally_ view the entire
> [code in GitHub](https://github.com/GoogleCloudPlatform/python-docs-samples/tree/master/people-and-planet-ai/image-classification).

In [ ]:
# We need libffi-dev to launch the Dataflow pipeline.
!apt-get -qq install libffi-dev

# Clone the python-docs-samples respository.
!git clone https://github.com/GoogleCloudPlatform/python-docs-samples.git

# Navigate to the sample code directory.
%cd python-docs-samples/people-and-planet-ai/image-classification

# Install the sample requirements.
!pip install --quiet -U pip
!pip install -r requirements.txt

> 🛎️ **\[DON’T PANIC\]** It’s safe to _ignore the warnings_.
> When we `pip install` the requirements, there might be some warnings about conflicting dependency versions.
> For the scope of this sample, that’s ok.

> ⚠️ **Restart the runtime**: Running the previous cell just updated some libraries and requires to restart the runtime to load those libraries correctly.
>
> In the top-left menu, click _**"Runtime" > "Restart runtime"**_.

In [ ]:
# After restarting the runtime, navigate back to the sample code directory.
%cd python-docs-samples/people-and-planet-ai/image-classification

### ✏️ Entering project details

Fill these in with the Google Cloud project resources you just created.

In [ ]:
google_cloud_project = "" #@param {type:"string"}
cloud_storage_bucket = "" #@param {type:"string"}
bigquery_dataset = "samples" #@param {type:"string"}

# Validate inputs.
if not google_cloud_project:
  raise ValueError('Please provide your google_cloud_project')
if not cloud_storage_bucket:
  raise ValueError('Please provide your cloud_storage_bucket')

# Authenticate to use the Google Cloud resources.
try:
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')
except ModuleNotFoundError:
  import os
  if os.environ.get('GOOGLE_APPLICATION_CREDENTIALS') is None:
    raise ValueError('Please set your GOOGLE_APPLICATION_CREDENTIALS environment variable to your service account JSON file path.')
  print(f"GOOGLE_APPLICATION_CREDENTIALS: {service_account_file}")

%env GOOGLE_CLOUD_PROJECT={google_cloud_project}

1. Click the run button ▶️ for the cells above.
1. Paste the "verification code" that is presented, and press **\[ENTER\]** to authenticate.

Run the following cell, you can leave them with these values or change them if you prefer.

In [ ]:
cloud_storage_directory = "samples/wildlife-insights" #@param {type:"string"}
bigquery_table = "wildlife_images_metadata" #@param {type:"string"}
ai_platform_name_prefix = "wildlife_classifier" #@param {type:"string"}
region = "us-central1" #@param {type:"string"}

> 🛎 Make sure the `region` matches the region you chose for your Cloud Storage bucket.

## 🐻 Creating the database of images metadata

To build our model we need data, so we create a database table from the metadata of all the images.
This is a _one-time only_ process.

Click run on the first Dataflow pipeline to create the images metadata table in BigQuery, which contains 2 fields of information from each image from the JSON file in WCS Camera Traps database:

   - `category`: The _species_ we want to predict, this is our _label_.
   - `file_name`: The _file path_ where the image file is located.
 
We do some very basic _data cleaning_ like discarding rows with categories that are note useful like `#ref!`, `empty`, `unidentifiable`, `unidentified`, `unknown`.

> 💡 As you collect new images from the field via your own camera traps, you continue to store the new image’s category and file name into the BigQuery table.
> You could extend this sample to collect more pertinent data for reporting purposes such as location, date, etc.

In [ ]:
# [One time only] Create the images metadata table.
!python create_images_metadata_table.py \
  --bigquery-dataset "{bigquery_dataset}" \
  --bigquery-table "{bigquery_table}" \
  --runner "DataflowRunner" \
  --job_name "wildlife-images-metadata-`date +%Y%m%d-%H%M%S`" \
  --project "{google_cloud_project}" \
  --temp_location "gs://{cloud_storage_bucket}/{cloud_storage_directory}/temp" \
  --region "{region}" \
  --worker_machine_type "n1-standard-2"

<button>

![View in GitHub](https://www.tensorflow.org/images/GitHub-Mark-32px.png)
[View `create_images_metadata_table.py`](https://github.com/GoogleCloudPlatform/python-docs-samples/blob/master/people-and-planet-ai/image-classification/create_images_metadata_table.py)

</button>

> 🛎️ We need at least `n1-standard-2` [worker machines](https://cloud.google.com/compute/docs/machine-types#n1_machine_types) due to large RAM usage when parsing the metadata JSON file.

You can look up the job details in the Dataflow jobs page:

- [console.cloud.google.com/dataflow/jobs](https://console.cloud.google.com/dataflow/jobs)

# 👟 Training the image classifier

One of the challenges of a camera trap dataset is that it usually contains a _different number of images per species_, making it very _unbalanced_.

For example, there are tens of thousands of pictures of some species like [tayassu pecari](https://www.google.com/search?q=tayassu+pecari&tbm=isch), while only a handful of pictures of other species like [tolypeutes matacus](https://www.google.com/search?q=tolypeutes+matacus&tbm=isch).

In _emojis_, the data looks _skewed_ like this: 

- 🦜🦜🦜🦜🦜 (5 Green Macaws)
- 🦧  (1 Bornean orangutan)
- 🐘🐘🐘 (3 Sumatran elephant)
- 🐗🐗🐗🐗🐗🐗🐗🐗🐗🐗🐗🐗 (12 wild boars)

This could introduce a
[_bias_](https://developers.google.com/machine-learning/crash-course/fairness/types-of-bias)
into your model which can lead to it predicting a species just because it's more common rather than identifying it by it’s actual features.
So instead of using the entire database of images, we leverage Dataflow to create a balanced dataset by specifying the _minimum_ and _maximum_ number of images we want for every category.
For categories with too many images, it chooses at most `max_images_per_class` randomly selected images.
And categories with less than `min_images_per_class` are discarded as _"too little information to teach our model to classify this species"_.

> 💡 For this sample, we defaulted to using _between 50 and 100 images_ per species/class to keep the training dataset small. This reduces the training time and the _potential cost_ of prediction accuracy.
> Feel free to play around with other numbers.

Once Dataflow chooses what images meet our criteria by counting the number of species per categories in BigQuery, it
[lazily](https://en.wikipedia.org/wiki/Lazy_evaluation)
downloads the actual JPEG images into Cloud Storage from the Lila Science database using it’s file name.
This means we are only preprocessing the images we need for training rather than processing the entire dataset of JPEG images.
 
When retraining in the future, if an image already exists in Cloud Storage, there's nothing else to do for that image, as we have created a _cache_ for images that have already been processed in our prior run.

After all the images are stored in Cloud Storage, Dataflow creates a
[CSV file for AI Platform](https://cloud.google.com/ai-platform-unified/docs/datasets/prepare-image#csv).
Each row includes the Cloud Storage path of an image, alongside with the category, our label.

Next, Dataflow tells AI Platform to import the CSV file and create a dataset from it. This is where the Dataflow job stops and AI Platform begins to train the model.
 
We let AI Platform do the _data splitting_ automatically for us. By default, it uses _80% of the data for training_, _10% for validation_, and _10% for testing_. See [About data splits for AI Platform models](https://cloud.google.com/ai-platform-unified/docs/general/ml-use) for more information.

> 🛎️ For simplicity, in this sample we are training a `CLOUD` model.
> This allows us to deploy it to an HTTP endpoint and get predictions via the browser.
>
> See [Train an Edge model](https://cloud.google.com/ai-platform-unified/docs/training/automl-edge-console)
> for information on how to train a model for a microcontroller.

In [ ]:
min_images_per_class = 50 #@param {type:"integer"}
max_images_per_class = 100 #@param {type:"integer"}

In [ ]:
# Create a balanced dataset and signal AI Platform to train a model.
!python train_model.py \
  --cloud-storage-path "gs://{cloud_storage_bucket}/{cloud_storage_directory}" \
  --bigquery-dataset "{bigquery_dataset}" \
  --bigquery-table "{bigquery_table}" \
  --ai-platform-name-prefix "{ai_platform_name_prefix}" \
  --min-images-per-class "{min_images_per_class}" \
  --max-images-per-class "{max_images_per_class}" \
  --runner "DataflowRunner" \
  --job_name "wildlife-train-model-`date +%Y%m%d-%H%M%S`" \
  --project "{google_cloud_project}" \
  --temp_location "gs://{cloud_storage_bucket}/{cloud_storage_directory}/temp" \
  --requirements_file "requirements.txt" \
  --region "{region}"

<button>

![View in GitHub](https://www.tensorflow.org/images/GitHub-Mark-32px.png)
[View `train_model.py`](https://github.com/GoogleCloudPlatform/python-docs-samples/blob/master/people-and-planet-ai/image-classification/train_model.py)

</button>

> 🛎️ It can take several minutes for the job to show up in the
> [Dataflow jobs page](https://console.cloud.google.com/dataflow/jobs).
> See [[ARROW-8983]](https://issues.apache.org/jira/browse/ARROW-8983) for more information.

Training the AI Platform model can take a while, depending on the dataset size and the training budget you allow.

> 💡 You can adjust the training budget using the `--budget-milli-node-hours` flag. We default to `8000` which is the minimum.
>
> See [the pricing page](https://cloud.google.com/vision/automl/pricing) and
> [`train_budget_milli_node_hours`](https://cloud.google.com/automl/docs/reference/rpc/google.cloud.automl.v1#imageclassificationmodelmetadata)
> for more information.

# 🐣 Deploying the model

> 🛎 Make sure your model has finished training.
> You can look at the training progress on the
> [AI Platform training jobs page](https://console.cloud.google.com/ai/platform/training/training-pipelines).

> 💡 If you were disconnected from the session due to inactivity, please make sure to re-run the cells at the _"Before you begin"_ section at the beginning of the notebook.

We need to deploy the trained model into an endpoint to get predictions from it.
Since we don't have a micro controller readily available, let’s deploy it in the cloud as an HTTP endpoint.

You can deploy it through the Cloud Console: [console.cloud.google.com/ai/platform/models](https://console.cloud.google.com/ai/platform/models)

Or alternatively, you can deploy it through the API by running the following cells:

In [ ]:
# First we need the model path, we can get it with gcloud.
# 💡 If you get an error here, please make sure your model has finished training.
cmd_output = !gcloud beta ai models list \
  --project {google_cloud_project} \
  --region {region} \
  --filter "display_name:{ai_platform_name_prefix}*" \
  --format "table[no-heading](display_name,name)" 2>/dev/null
models = sorted([line.split() for line in cmd_output])
model_path = models[0][1]

print(f"model_path: {model_path}")

In [ ]:
# Create an endpoint and deploy the model to it.
!python deploy_model.py \
  --project {google_cloud_project} \
  --region {region} \
  --model-path {model_path} \
  --model-endpoint-name {ai_platform_name_prefix}

<button>

![View in GitHub](https://www.tensorflow.org/images/GitHub-Mark-32px.png)
[View `deploy_model.py`](https://github.com/GoogleCloudPlatform/python-docs-samples/blob/master/people-and-planet-ai/image-classification/deploy_model.py)

</button>

# 📺 Visualizing classification of images online

Now that we have a deployed model, we can classify images using it.
 
First, lets define some functions to help us visualize and navigate the images, click run:

In [ ]:
import io
import requests
from PIL import Image
from IPython.display import display

from google.cloud import bigquery


def display_image(image_file, width=400):
  base_url = 'https://lilablobssc.blob.core.windows.net/wcs-unzipped'
  image_bytes = requests.get(f"{base_url}/{image_file}").content
  if b'<Error>' in image_bytes:
    raise ValueError(f"Error requesting image: {base_url}/{image_file}\n{image_bytes.decode('utf-8')}")
  image = Image.open(io.BytesIO(image_bytes))
  display(image.resize((int(width), int(width / image.size[0] * image.size[1]))))


def display_samples_for_category(category, num_samples=3, width=400):
  client = bigquery.Client()
  query_job = client.query(f"""
      SELECT file_name
      FROM `{google_cloud_project}.{bigquery_dataset}.{bigquery_table}`
      WHERE category = '{category}'
      LIMIT {num_samples}
  """)

  for row in query_job:
    image_file = row['file_name']
    print(f"{category}: {image_file}")
    display_image(image_file, width)

Now run the following cell to get the model endpoint ID, which we need to make predictions.

In [ ]:
# First we need the endpoint ID, we can get it with gcloud.
stdout = !gcloud beta ai endpoints list \
  --project {google_cloud_project} \
  --region {region} \
  --filter "display_name={ai_platform_name_prefix}" \
  --format "table[no-heading](ENDPOINT_ID)" 2>/dev/null
model_endpoint_id = stdout[0]

print(f"model_endpoint_id: {model_endpoint_id}")

In [ ]:
def predict(image_file):
  # First, display the image, and then we run our prediction script.
  display_image(image_file)
  !python predict.py \
    --project "{google_cloud_project}" \
    --region "{region}" \
    --model-endpoint-id "{model_endpoint_id}" \
    --image-file "{image_file}"

<button>

![View in GitHub](https://www.tensorflow.org/images/GitHub-Mark-32px.png)
[View `predict.py`](https://github.com/GoogleCloudPlatform/python-docs-samples/blob/master/people-and-planet-ai/image-classification/predict.py)

</button>

Now, run the following cells to see what our model thinks about each of the following images.
The model gives us a list of species it predicts it is, alongside with the confidence score for each prediction.

In [ ]:
# Species: dicerorhinus sumatrensis
predict('animals/0325/1529.jpg')

In [ ]:
# Species: didelphis imperfecta
predict('animals/0667/1214.jpg')

In [ ]:
# Species: tapirus indicus
predict('animals/0036/0072.jpg')

In [ ]:
# Species: leopardus wiedii
predict('animals/0000/1705.jpg')

In [ ]:
# Species: hemigalus derbyanus
predict('animals/0036/0566.jpg')

In [ ]:
# Species: dasypus novemcinctus
predict('animals/0000/0425.jpg')

🕵️ While analyzing the model’s evaluation, you may notice that many species of the same family are very similar and it may be confusing the model. This means it might be worth trying to classify the image to a family instead of a specific species, or get more images from different angles/proximity to detect it better.

# 🧹 Cleaning up to avoid additional costs

To avoid incurring charges to your Google Cloud account for the resources used in this tutorial, either delete the project that contains the resources, or keep the project and delete the individual resources.
 
If you wish to delete the project you created for this tutorial (it’s the easiest way to eliminate billing), here are the steps:

## Deleting the project

The easiest way to eliminate billing is to delete the project that you created for the tutorial.

To delete the project:

> ⚠️ **Caution**: Deleting a project has the following effects:
>
> - **Everything in the project is deleted.** If you used an existing project for this tutorial, when you delete it, you also delete any other work you've done in the project.
> - **Custom project IDs are lost.** When you created this project, you might have created a custom project ID that you want to use in the future. To preserve the URLs that use the project ID, such as an `appspot.com` URL, delete selected resources inside the project instead of deleting the whole project.
>
> If you plan to explore multiple tutorials and quickstarts, reusing projects can help you avoid exceeding project quota limits.

1. In the Cloud Console, go to the **Manage resources** page.

  <button>

  [Go to Manage resources](https://console.cloud.google.com/iam-admin/projects)

  </button>

1. In the project list, select the project that you want to delete, and then click **Delete**.

1. In the dialog, type the project ID, and then click **Shut down** to delete the project.


## 🎉 Congratulations, you have successfully completed this code lab!
 
🙌 You made it this far, congrats.
Dive deeper by visiting the
[code on GitHub](https://github.com/GoogleCloudPlatform/python-docs-samples/tree/master/people-and-planet-ai/image-classification).

> 🛎️ Was this a meaningful project for you? If so you can you share your experience in
> [these 2 quick questions here](https://docs.google.com/forms/d/e/1FAIpQLSeClJev12m2CGE5uwJAvfJRZe9LYUZR4nimsDjrRjvc5xBifA/viewform). 